In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

# Final Project: Part 1
Contributions From: Ryan Cottone

Welcome to the final project of the semester! In Part 1, we will use what we learned about cryptography to recreate the Transport Layer Security (TLS) protocol that is used to encrypt your connections to websites. We will also build a server authentication and file storage system.

In Part 2, you will be presented with ~8 different challenges of varying difficulty. Each challenge will involve finding and exploiting some flaw in the setup of the aforementioned system.

Please see the spec (TODO: LINK HERE) for a guideline on completing this part.

### Helpers

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
%%capture
import random
import math
import string
import numpy as np
from utils import *

In [ ]:
CERT_AUTH = { 'e': 65537, 'N': 33790420484761320225234266446986435791020053290995177788399417698648848366075439013295931744537889745793682732187585867814285806211190774412138926826806937374931229955338241741978503726324443629746710612128866806815968501932728765477787763877641403710570749219182260822344263730489611164845428107854720086677, 'd': 13990616901200824332998639242549657982350872729162978917073076266121984534132734754806980909837734993242162151729712109543321259006983256231951260806039426156135728347335452488348561573956342300497866864117403272359970143321087093698235850894812096199039329431651823513521315154736696406982035194667449777653}

# Transport Layer Security

First: what even is TLS? TLS is a protocol designed to securely encrypt communications between a client and a web server. It uses a combination of public-key encryption, symmetric encryption, digital signatures, and more!

We will be using the **RSA-ECDH** version of TLS. In order, the client and server accomplish the following steps:

Before the connection begins, the server needs to configure certain constants and get a certificate signed by a certificate authority.
1. The server generates an RSA keypair of the form ((e, N), d).
2. The certificate authority issues a signature saying the server's RSA public key corresponds to that server.
3. The server picks an elliptic curve (we will use the Sepc256k1 curve) and a generator point.

After the server is setup, the client and server initiate a connection and trade RSA keys.
1. The client asks the server for its RSA public key and certificate.
2. The server provides this, and the client verifies the certificate is valid for that RSA key.
3. The server then sends the elliptic curve parameters, signed by their RSA private key.
4. The client verifies these parameters are correctly signed by the server's RSA public key.

Now that the client and server have securely traded RSA keys, they are able to sign messages to ensure integrity and authenticity. We can begin the Elliptic-Curve Diffie Hellman Key Exchange (ECDH).
1. The server generates a secret value a and public point aP using the aforementioned generator point, and sends AP to the client. They attach a signature of aP using their RSA private key.
2. The client receives nP, and computes their secret b. They compute bP and send it to the server (alongside a signature on bP).
3. The server computes a(bP) = (ab)P and the client computes b(aP) = (ab)P. They now convert this point to an integer and use it as a shared secret.

Once we have securely established a shared secret, we can use symmetric encryption to send requests. 
1. The client create requests and sends it encrypted via AES using the shared secret derived from ECDH. The server can decrypt this, and encrypt its response using the same method.

In [ ]:
class Server:
    RSA_PUBLIC_KEY = (
        65537,
        9038494040587010144527283006157928881319808198037800262507236135386268739708006022679916006469762927636026303078260115925689450875819797050237830498287577
                     )
    
    RSA_PRIVATE_KEY = 2535417770757764232372393774283341693305817384267313426399332119458339022396333163725804069143378584578179197504985944189947045947523212247400435285708353
    
    SIGNED_SERVER_CERTIFICATE = 12700340844416036644598467941678481034795616611285372846483555754936821137468034013552250333992213649482476234812345812443409067513447778032927158222423442969388095714455462826384483204764801308373583214095146933570191105218603640338334545613351980554082339092228528550094782514990171966181657825617151691397
    
    ELLIPTIC_CURVE = [0,7]
    
    ELLIPTIC_CURVE_MODULUS = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
    
    GENERATOR_POINT = (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
    
    GENERATOR_POINT_ORDER = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
    
    LOGINS = {}

    def signMessage(self, message):
        """
        Helper function to sign a given (integer) message with this instance's private key.
        """
        
        return signRSA(self.RSA_PRIVATE_KEY, message, self.RSA_PUBLIC_KEY[1])
    
    def publishRSAPublicKey(self):
        """
        Returns ((e, N), signature) where e,N are the server's public key, and signature is the cert authorities' signature 
        on e||N.
        """
        
        return ((self.RSA_PUBLIC_KEY[0], self.RSA_PUBLIC_KEY[1]), self.SIGNED_SERVER_CERTIFICATE)
    
    def generateECDHParameters(self):
        """
        Returns a generator point P, curve E, and modulus N signed via the server's RSA keys.
        """

        intToSign = int(str(self.ELLIPTIC_CURVE[0]) + str(self.ELLIPTIC_CURVE[1]) 
                        + str(self.ELLIPTIC_CURVE_MODULUS) + str(self.GENERATOR_POINT[0]) + str(self.GENERATOR_POINT[1]))

        signature = ...

        return self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT, self.GENERATOR_POINT_ORDER, signature
    
    def generateSignedECDHMessage(self):
        """
        Helper function that generates a random ECDH keypair (nP, n) and returns nP, signature where signature is the 
        RSA signature on nP.
        """
        
        ...
        
        intToSign = int(str(nP[0]) + str(nP[1]))
        
        self.ECDH_SECRET = ...
        
        return nP, self.signMessage(intToSign)
    
    def verifyECDHMessage(self, P, signature, publicKey):
        """
        Verifies point P was signed by the given publicKey (e, N).
        """

        expected = int(str(P[0]) + str(P[1]))

        ...
    
    def acceptConnection(self, clientPublicKey):
        self.CLIENT_PUBLIC_KEY = clientPublicKey
    
    def acceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        
        assert self.verifyECDHMessage(P, signature, self.CLIENT_PUBLIC_KEY), "Invalid signature on received ECDH message."
        
        # Signature verified
        sharedSecretPoint = ...
        
        self.SHARED_SECRET_INT = ...
        
        return True
    
    def createLogin(self, username, password):
        """
        Given a username and password, update the internal LOGINS database to have an entry for username.
        """
        
        if username in self.LOGINS: 
            return False
        
        salt = ''.join(np.random.choice(list(string.ascii_lowercase), 5))
        
        intToHash = textToInt(password + salt)
        
        hashedPassword = ...
        
        
        self.LOGINS[username] = (hashedPassword, salt)
        
        return True
        
    def verifyLogin(self, username, password):
        """
        Given a username and password, check if the password corresponds to the given username.
        """
        
        assert username in self.LOGINS, "Username does not exist"
        
        hashedPassword = ...
        salt = ...
        
        intToHash = textToInt(password + salt)
                
        return hashedPassword == hash_SHA256(intToHash)
    
    def encryptMessage(self, message):
        """
        Given string message, encrypts with the current setup
        """
        
        return encryptAES(self.SHARED_SECRET_INT, bytes(message))
    
    def decryptMessage(self, encrypted):
        """
        Given encrypted message, decrypts with the current setup
        """
        
        return decryptAES(self.SHARED_SECRET_INT, encrypted)
    
    def verifyRequestIntegrity(self, requestedData):
        """
        Takes in a request of the form { data: ..., hmac: ...} and returns True if the HMAC is valid.
        """
        
        ...
    
    def handleRequest(self, requestData):
        """
        Handles the raw request data and routes it to the correct function.
        """
        
        if 'data' not in requestData or 'hmac' not in requestData:
            raise AssertionError("Invalid requested format")
            
        assert self.verifyRequestIntegrity(requestData), "Request MAC check failed"
        
        # Request verified, time to decrypt
        request = json.loads(self.decryptMessage(requestData['data']))
        
        if request['type'] == 'createLogin':
            assert 'username' in request and 'password' in request, "Bad request data"
            
            return self.createLogin(request['username'], request['password'])
        elif request['type'] == 'verifyLogin':
            return self.verifyLogin(request['username'], request['password'])
        else:
            raise AssertionError("Invalid request type")

In [ ]:
class Client:
    def __init__(self, keybits=512):
        rsa_keypair = generateRSAKeypair(keybits)
        
        self.RSA_PUBLIC_KEY = (rsa_keypair[0], rsa_keypair[1])
        self.RSA_PRIVATE_KEY = (rsa_keypair[2])
        
    def signMessage(self, message):
        """
        Helper function to sign a given (integer) message with this instance's private key.
        """
        
        return signRSA(self.RSA_PRIVATE_KEY, message, self.RSA_PUBLIC_KEY[1])
        
    def verifyServerCertificate(self, serverPublicKey, signature):
        """
        Asserts the server's certificate is correctly signed by the given certificate authority.

        The certificate is on int(e||N).
        """
        
        e, N = CERT_AUTH['e'], CERT_AUTH['N']

        ...
            return False
        
        self.SERVER_PUBLIC_KEY = serverPublicKey
        
        return True
    
    def verifyECDHParameters(self, ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature):
        """
        Verifies the ECDH parameters sent by the server were correctly signed, and accepts them if so.
        """
        
        intToSign = int(str(ellipticCurve[0]) 
                        + str(ellipticCurve[1])
                        + str(ellipticCurveModulus) 
                        + str(generatorPoint[0]) 
                        + str(generatorPoint[1]))

        ...
            return False
        
        self.ELLIPTIC_CURVE = ellipticCurve
        self.ELLIPTIC_CURVE_MODULUS = ellipticCurveModulus
        self.GENERATOR_POINT = generatorPoint
        self.GENERATOR_POINT_ORDER = generatorPointOrder
        
        return True
    
    
    def generateSignedECDHMessage(self):
        """
        Helper function that generates a random ECDH keypair (nP, n) and returns nP, signature where signature is the 
        RSA signature on nP.
        Assumes we have already setup a valid connection.
        """
        
        
        ...
        
        intToSign = int(str(nP[0]) + str(nP[1]))
        
        self.ECDH_SECRET = ...
        
        return nP, self.signMessage(intToSign)
    
    def verifyECDHMessage(self, P, signature, publicKey):
        """
        Verifies point P was signed by the given publicKey (e, N).
        """

        expected = int(str(P[0]) + str(P[1]))

        ...
    
    def acceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        
        ...
        
        # Signature verified
        sharedSecretPoint = ...
        
        self.SHARED_SECRET_INT = pointToMessage(sharedSecretPoint)
        
        return True
    
    def encryptMessage(self, message):
        """
        Given string message, encrypts with the current setup
        """
        
        return encryptAES(self.SHARED_SECRET_INT, bytes(message, 'utf-8'))
    
    def decryptMessage(self, encrypted):
        """
        Given encrypted message, decrypts with the current setup
        """
        
        return decryptAES(self.SHARED_SECRET_INT, encrypted)
    
    def generateRequest(self, obj):
        """
        Helper function to turn an object into a request wrapper.
        """
        
        encStr = self.encryptMessage(json.dumps(obj))
        
        hmac = generateHMAC(self.SHARED_SECRET_INT, bytes(encStr, 'utf-8'))
        
        requestObj = {'data': encStr, 'hmac': hmac}
        
        return requestObj

**Question 1.1:** Implement Client.verifyServerCertificate.

In [ ]:
grader.check("q1_1")

**Question 1.2:** Implement Server.generateECDHParameters in the Server class above. Run the below test when you are finished.

In [ ]:
grader.check("q1_2")

**Question 1.3:** Implement Client.verifyECDHParameters

In [ ]:
grader.check("q1_3")

**Question 1.4:** Implement generateSignedECDHMessage in both Server and Client classes.

In [ ]:
grader.check("q1_4")

**Question 1.5:** Implement acceptECDHMessage in both Server and Client.

In [ ]:
grader.check("q1_5")

**Question 1.6:** Implement Client.generateRequest

In [ ]:
grader.check("q1_6")

**Question 1.7:** Implement Server.verifyRequestIntegrity

In [ ]:
grader.check("q1_7")

**Question 1.8:** Implement Server.createLogin

In [ ]:
grader.check("q1_8")

**Question 1.9:** Implement Server.verifyLogin

In [ ]:
grader.check("q1_9")

Congrats on finishing Part 1 of this project! Here is an example integration test:

In [ ]:
# New client
client = Client()
server = Server()
server.acceptConnection(client.RSA_PUBLIC_KEY)

server_PK, signature = server.publishRSAPublicKey()
assert client.verifyServerCertificate(server_PK, signature)

# Once verified, ask for ECDH parameters
ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()
assert client.verifyECDHParameters(ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature)

# Once ECDH parameters are verified, generate our secret + public value
client_ECDH = client.generateSignedECDHMessage()
server_ECDH = server.generateSignedECDHMessage()

# client gets server ecdh 
client.acceptECDHMessage(server_ECDH[0], server_ECDH[1])

# server gets client ecdh
server.acceptECDHMessage(client_ECDH[0], client_ECDH[1])
assert client.SHARED_SECRET_INT == server.SHARED_SECRET_INT

signupRequest = client.generateRequest({'type': 'createLogin', 'username': 'asciabsjf', 'password': 'aidojsd1r'})
server.handleRequest(signupRequest)

loginRequest = client.generateRequest({'type': 'verifyLogin', 'username': 'asciabsjf', 'password': 'aidojsd1r'})
assert server.handleRequest(loginRequest)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)